In [ ]:
import csv
from math import radians, sin, cos, sqrt, atan2
import time
import bisect


import csv
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial import KDTree


In [3]:

# --- Haversine distance function ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# --- Load CSV ---


In [197]:
entries1 = []
with open("p7.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for row in reader:
        lat = int(row[0]) / 1e6
        lon = int(row[1]) / 1e6
        name = row[2]
        entries1.append((lat, lon, name))

entries2 = []

with open("p7_sorted.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for row in reader:
        lat = int(row[0]) / 1e6
        lon = int(row[1]) / 1e6
        name = row[2]
        entries2.append((lat, lon, name))

entries3 = []   # (lat, lon, name)
coords = []    # (lat, lon) only for KDTree
with open("p7.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for row in reader:
        name = row[2]
        lat = float(row[0]) / 1e6   # adjust scaling
        lon = float(row[1]) / 1e6
        entries3.append((lat, lon, name))
        coords.append((lat, lon))

        


In [9]:

# --- Function to find nearest place ---
def find_nearest(lat_gps, lon_gps):
    min_dist = float('inf')
    nearest_name = None
    for lat, lon, name in entries1:
        d = haversine(lat_gps, lon_gps, lat, lon)
        if d < min_dist:
            min_dist = d
            nearest_name = name
    return nearest_name, min_dist


In [164]:
lats = [lat for lat, _, _ in entries2]

def find_nearest_bin(lat_gps, lon_gps):
    # Extract only latitudes for binary search

    # Position to insert given latitude
    idx = bisect.bisect_left(lats, lat_gps)

    # Define search window (e.g., 0.5° latitude ≈ 55 km)
    delta = 0.005
    lat_min, lat_max = lat_gps - delta, lat_gps + delta

    # Find bounds in sorted array
    lo = bisect.bisect_left(lats, lat_min)
    hi = bisect.bisect_right(lats, lat_max)

    print(f"Searching between indices {lo} and {hi} out of {len(entries2)} entries")

    # Search only inside this range
    min_dist = float('inf')
    nearest_name = None
    for lat, lon, name in entries2[lo:hi]:
        d = haversine(lat_gps, lon_gps, lat, lon)
        if d < min_dist:
            min_dist = d
            nearest_name = name

    return nearest_name, min_dist

In [198]:
def find_nearest_kd(lat_gps, lon_gps):
    # Query nearest neighbor (by Euclidean distance in lat/lon space)
    dist, idx = tree.query((lat_gps, lon_gps), k=1)
    
    # Extract nearest point
    lat, lon, name = entries3[idx]
    
    # Correct distance with haversine
    d_m = haversine(lat_gps, lon_gps, lat, lon)
    
    return name, d_m

In [183]:

gps_lat, gps_lon = 26.506447, 80.232849



nearest_name, distance_m = find_nearest_bin(gps_lat, gps_lon)
print(f"Nearest named place: {nearest_name} ({distance_m:.1f} meters away)")


Searching between indices 66687 and 68130 out of 108553 entries
Nearest named place: Hall of Residence for Girls 1 (0.9 meters away)


In [224]:
tree = KDTree(coords)


In [228]:
start1 = time.perf_counter()
name1, d1 = find_nearest_bin(gps_lat, gps_lon)
end1 = time.perf_counter()

start2 = time.perf_counter()
name2, d2 = find_nearest(gps_lat, gps_lon)
end2 = time.perf_counter()

start3 = time.perf_counter()
name3, d3 = find_nearest_kd(gps_lat, gps_lon)
end3 = time.perf_counter()

print(f"[With bisect]   {name1} ({d1:.1f} m)  → {(end1-start1)*1000:.3f} ms")
print(f"[Full scan]    {name2} ({d2:.1f} m)  → {(end2-start2)*1000:.3f} ms")
print(f"[With KDTree]  {name3} ({d3:.1f} m)  → {(end3-start3)*1000:.3f} ms")

Searching between indices 66687 and 68130 out of 108553 entries
[With bisect]   Hall of Residence for Girls 1 (0.9 m)  → 1.576 ms
[Full scan]    Hall of Residence for Girls 1 (0.9 m)  → 10.254 ms
[With KDTree]  Hall of Residence for Girls 1 (0.9 m)  → 0.320 ms
